In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [0]:
# name_schema = StructType(fields=[StructField("forename", StringType(), True),
#                                   StructField("surname", StringType(), True)])
                    

In [0]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                     StructField("driverRef", StringType(), True),
                                     StructField("number", IntegerType(), True),
                                     StructField("code", StringType(), True),
                                     StructField("forename", StringType(), True),
                                     StructField("surname", StringType(), True),
                                     StructField("dob", StringType(), True),
                                     StructField("nationality", StringType(), True),
                                     StructField("url", StringType(), False)])

In [0]:
drivers_df = spark.read.schema(drivers_schema).csv(f"{raw_folder_path}/{v_file_date}/drivers.csv")

In [0]:
from pyspark.sql.functions import col,concat,current_timestamp,lit

In [0]:
drivers_with_columns_df = add_ingestion_date(drivers_df) \
    .withColumnRenamed("driverId","driver_id") \
                                    .withColumnRenamed("driverRef","driver_ref") \
                                    .withColumn("name",concat(col("forename"),lit(" "),col("surname")))\
.withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date)) \
        .drop("forename") \
        .drop("surname")


In [0]:
drivers_final_df = drivers_with_columns_df.drop("url")

In [0]:
display(drivers_final_df)

In [0]:
drivers_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.drivers")

In [0]:
dbutils.notebook.exit("success")